In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset,concatenate_datasets
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, GPT2Model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from bayes_opt import BayesianOptimization
import numpy as np
import random

2024-08-09 16:45:26.355167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 16:45:26.355292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 16:45:26.492458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# set the random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    
# Load the dataset
dataset = load_dataset("SetFit/bbc-news")

# Convert the training and test datasets to DataFrames
train_df = dataset['train'].to_pandas()
test_df = dataset['test'].to_pandas()

# Combine the training and test DataFrames
all_df = pd.concat([train_df, test_df])

# 30 % data are used for test
train_df, test_df = train_test_split(all_df, test_size=0.2, random_state=seed)

#  20% of training set are divided to be validation set
train_df, val_df = train_test_split(train_df, test_size=0.125, random_state=seed)

# turn DataFrames into Dataset object
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


Generating train split:   0%|          | 0/1225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
# Load GPT2Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')



# Add pad_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2ForSequenceClassification.from_pretrained('distilgpt2', num_labels=5)

# Resize token embeddings to match the tokenizer length
model.resize_token_embeddings(len(tokenizer))

# Set the padding token ID in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

train_data1 = train_dataset.map(tokenize_function, batched=True)
val_data1 = val_dataset.map(tokenize_function, batched=True)
test_data1 = test_dataset.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed=seed,
)

# Define Trainer
trainer1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data1,
    eval_dataset=val_data1,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer1.train()

# Evaluate the best model on the test set
eval_result = trainer1.evaluate(test_data1)
print(f"Final evaluation results on test set: {eval_result}")

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.357079,0.932735,0.932773,0.941962,0.932735
2,No log,0.164550,0.973094,0.972913,0.973114,0.973094
3,No log,0.202780,0.973094,0.972906,0.973465,0.973094
4,No log,0.134313,0.973094,0.973002,0.973184,0.973094
5,No log,0.172744,0.973094,0.973006,0.973118,0.973094
6,0.274400,0.162155,0.973094,0.973002,0.973184,0.973094
7,0.274400,0.195784,0.959641,0.959616,0.960731,0.959641
8,0.274400,0.189557,0.973094,0.972998,0.973309,0.973094
9,0.274400,0.196277,0.973094,0.972998,0.973309,0.973094
10,0.274400,0.194732,0.973094,0.972998,0.973309,0.973094


Final evaluation results on test set: {'eval_loss': 0.15738895535469055, 'eval_accuracy': 0.9707865168539326, 'eval_f1': 0.9708435085090811, 'eval_precision': 0.9715983068651995, 'eval_recall': 0.9707865168539326, 'eval_runtime': 4.5301, 'eval_samples_per_second': 98.231, 'eval_steps_per_second': 6.181, 'epoch': 10.0}


In [6]:
enhanced_prompts = [
"What is the central focus of this newspaper article?",
"How would you formulate the main theme of this message?",
"Where is this story most likely to be found in the newspaper?",
"How would you define the primary subject of this news?",
"What is the main topic of this news?",
"Which category is best suited for this news?",
"What is the main topic of this news article?",
"What is central to this news article?",
"Which part of the newspaper is most likely to contain this news?",
"Where in the newspaper is this news most likely to be located?",
"How would you explain the main topic of this news?",
"Which section of the newspaper is this news most likely to be found in?",
"In what section of the newspaper is this news most likely to be found?",
"Which category is the best match for this news article?",
"In which category does this news article best fit?",
"Which category does this news fall into?",
"Which category is most appropriate for this news?",
"What category does this news belong to?",
"Which section of the newspaper will most likely contain this news?",
"In which section of the newspaper is this news story most likely to appear?",
"What is the central purpose of this news article?",
"In which section of the newspaper would one likely find this news item?",
"How would you summarize the central theme of this message?",
"In which section of the newspaper is this news most likely?",
"Which part of the newspaper is likely to contain this news?",
"Which category does this news best fit into?",
"How would you define the main topic of this news?",
"Which category best matches this news article?",
"What is the main emphasis of this news article?",
"How would you describe the main topic of this news?",
"In which section of the newspaper would this news item most likely be found?",
"How would you characterize the main theme of this message?",
"Where in the newspaper is this news story most likely to be found?",
"What is your description of the main theme of this news?",
"Which category is best for this news article?",
"What is the main point of this press article?",
"In your opinion, what is the main objective of this news?",
"In which section of the newspaper is this news most likely to appear?",
"What category does this news article fall under?",
"In what section of the newspaper is this news most likely to appear?",
"Under which category does this news article fit best?",
"What category does this news article belong to?",
"Where in the newspaper is this news most likely to appear?",
"What is your description of the main topic of this news story?",
"In which section of the newspaper would this news most likely be found?",
"How would you describe the main theme of this message?",
"Which section of the newspaper is most likely to contain this news?",
"Where in the newspaper is this news most likely to be found?",
"In which section of the newspaper is this news most likely to be found?",
"In which section of the newspaper are you most likely to find this news?",
"What is this news article focused on?",
"In what part of the newspaper would this news most likely appear?",
"How would you articulate the primary subject of this news?",
"What do you think is the main focus of this message?",
"What is the main emphasis of this news?",
"How would you define the main theme of this message?",
"Which part of the newspaper would most likely contain this news?",
"Which category best fits this news?",
"How would you explain the main theme of this news?",
"Which category is best suited for this news article?",
"What do you think is the main focus of this news?",
"Which category does this news article best fit into?",
"How would you describe the main topic of this news story?",
"What is the main focus of this news?",
"In which part of the newspaper is this news likely to be found?",
"In which category does this news article belong?",
"Which category best suits this news article?",
"What do you think is the main objective of this news?",
"How would you summarize the central theme of this news?",
"What would you say is the primary focus of this news?",
"How do you describe the main topic of this news story?",
"How would you characterize the main topic of this news story?",
"How would you describe the main theme of this news story?",
"What is the primary focus of this news article?",
"How would you define the primary topic of this news item?",
"Which section of the newspaper is most likely to publish this story?",
"What is the main focus of this news article?",
"What is the main theme of this news?",
"In which category does this news article fit best?",
"What is your description of the main subject of this news?",
"What is your description of the main topic of this news?",
"How would you articulate the main subject of this news?",
"How would you explain the main subject of this news?",
"What category does this newspaper article belong to?",
"Which section of the newspaper is most likely to contain this news story?",
"How would you phrase the main subject of this news story?",
"Where in the newspaper might this news be found?",
"What is the central point of this news article?",
"Which category best fits this news article?",
"What is the main purpose of this press article?",
"What is the main subject of this news article?",
"How would you characterize the main subject of this news?",
"Which section of the newspaper is most likely to feature this news?",
"In which part of the newspaper is this news most likely to be found?",
"In which section of the newspaper is it most likely to find this news?",
"In which part of the newspaper is this news most likely to appear?",
"What is the main topic of this article?",
"What category does this news article fall into?",
"Into which category does this news article best fit?",
"Which category is most suitable for this news article?",
"How would you phrase the primary topic of this news story?",
"In which section of the newspaper would this news likely be found?",
"What would you say is the main focus of this news?",
"How would you explain the main theme of this message?",
"In which part of the newspaper would this news most likely appear?",
"Which category does this news article fall under?",
"How would you describe the main theme of this news?",
"What is the main point of this news?",
"What is the key focus of this news article?",
"In which section of the newspaper would this news story likely be found?",
"How would you articulate the main point of this news story?",
"Which part of the newspaper would most likely contain this news story?",
"What is the central focus of this news article?",
"What is the main point of this news article?",
"What category is this news article in?",
"How would you characterize the main topic of this news?",
"Under which category does this news article best fit?",
"What category best fits this news article?",
"Which category is the best match for this news story?",
"Which section of the newspaper is this news most likely to be in?",
"How would you define the main subject of this news?",
"Which part of the newspaper would most likely feature this news?",
"In which section of the newspaper is this news story most likely to be found?",
"Which category does this news article best fit under?",
"How would you summarize the central theme of this news story?",
"What is the focus of this news article?",
"How would you articulate the main theme of this news story?",
"Which category is most appropriate for this news article?",
"How would you explain the main topic of this news story?",
"Which category does this news article belong to?",
"In which section of the newspaper is this news most likely to be published?",
"How would you describe the main subject of this news?",
"How would you express the main topic of this news?",
"What is the central focus of this news?",
"How would you articulate the main topic of this news?"
]

enhanced_prompts1= [ 
"What category best fits this news article?",
"How would you describe the main topic of this news?",
"In which section of the newspaper would this news likely be found?",
"What is the primary focus of this news article?",
"Which category does this news article fall under?",
"In which category does this news article belong?",
"Which category is most appropriate for this news article?",
"Under which category does this news article best fit?",
"Which category is the best match for this news article?",
"Into which category does this news article best fit?",
"Which category is most suitable for this news article?",
"How would you define the primary subject of this news?",
"How would you summarize the central theme of this news?",
"What would you say is the primary focus of this news?",
"How would you explain the main subject of this news?",
"How would you characterize the main topic of this news?",
"What is your description of the main topic of this news?",
"How would you articulate the primary subject of this news?",
"Which section of the newspaper is most likely to feature this news?",
"In what section of the newspaper is this news most likely to appear?",
"Which section of the newspaper is this news most likely to be found in?",
"Which part of the newspaper would most likely feature this news?",
"Which section of the newspaper is most likely to contain this news?",
"In what section of the newspaper is this news most likely to be found?",
"Where in the newspaper is this news most likely to be located?",
"What is the main emphasis of this news article?",
"What is the central focus of this news article?",
"What is the main point of this news article?",
"What is the main subject of this news article?",
"What is the main focus of this news article?",
"What is the key focus of this news article?",
"What is the main topic of this news article?",
"What category best fits this news article: business, entertainment, politics, sport or tech?",
"How would you describe the main topic of this news: business, entertainment, politics, sport or tech?",
"In which section of the newspaper would this news likely be found: business, entertainment, politics, sport or tech?",
"What is the primary focus of this news article: business, entertainment, politics, sport or tech?",
"Which category does this news article fall under: business, entertainment, politics, sport or tech?",
"In which category does this news article belong: business, entertainment, politics, sport or tech?",
"Which category is most appropriate for this news article: business, entertainment, politics, sport or tech?",
"Under which category does this news article best fit: business, entertainment, politics, sport or tech?",
"Which category is the best match for this news article: business, entertainment, politics, sport or tech?",
"Into which category does this news article best fit: business, entertainment, politics, sport or tech?",
"Which category is most suitable for this news article: business, entertainment, politics, sport or tech?",
"How would you define the primary subject of this news: business, entertainment, politics, sport or tech?",
"How would you summarize the central theme of this news: business, entertainment, politics, sport or tech?",
"What would you say is the primary focus of this news: business, entertainment, politics, sport or tech?",
"How would you explain the main subject of this news: business, entertainment, politics, sport or tech?",
"How would you characterize the main topic of this news: business, entertainment, politics, sport or tech?",
"What is your description of the main topic of this news: business, entertainment, politics, sport or tech?",
"How would you articulate the primary subject of this news: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is most likely to feature this news: business, entertainment, politics, sport or tech?",
"In what section of the newspaper is this news most likely to appear: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is this news most likely to be found in: business, entertainment, politics, sport or tech?",
"Which part of the newspaper would most likely feature this news: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is most likely to contain this news: business, entertainment, politics, sport or tech?",
"In what section of the newspaper is this news most likely to be found: business, entertainment, politics, sport or tech?",
"Where in the newspaper is this news most likely to be located: business, entertainment, politics, sport or tech?",
"What is the main emphasis of this news article: business, entertainment, politics, sport or tech?",
"What is the central focus of this news article: business, entertainment, politics, sport or tech?",
"What is the main point of this news article: business, entertainment, politics, sport or tech?",
"What is the main subject of this news article: business, entertainment, politics, sport or tech?",
"What is the main focus of this news article: business, entertainment, politics, sport or tech?",
"What is the key focus of this news article: business, entertainment, politics, sport or tech?",
"What is the main topic of this news article: business, entertainment, politics, sport or tech?",
"What category best fits this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"How would you describe the main topic of this news: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"In which section of the newspaper would this news likely be found: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the primary focus of this news article: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"Which category does this news article fall under: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"In which category does this news article belong: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which category is most appropriate for this news article: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Under which category does this news article best fit: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which category is the best match for this news article: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Into which category does this news article best fit: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"Which category is most suitable for this news article: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"How would you define the primary subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"How would you summarize the central theme of this news: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"What would you say is the primary focus of this news: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"How would you explain the main subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"How would you characterize the main topic of this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"What is your description of the main topic of this news: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"How would you articulate the primary subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Which section of the newspaper is most likely to feature this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"In what section of the newspaper is this news most likely to appear: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which section of the newspaper is this news most likely to be found in: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Which part of the newspaper would most likely feature this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Which section of the newspaper is most likely to contain this news: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"In what section of the newspaper is this news most likely to be found: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Where in the newspaper is this news most likely to be located: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the main emphasis of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the central focus of this news article: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"What is the main point of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the main subject of this news article: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"What is the main focus of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the key focus of this news article: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"What is the main topic of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport." ]

In [7]:
# classify function
def classify_article_with_prompt(prompt, article_text):
    input_text = f"{prompt} {article_text}"
    inputs = tokenizer(input_text, truncation=True, max_length=512, padding='max_length', return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    label_id = outputs.logits.argmax(dim=1).item()
    return label_id

# evaluation function
def evaluate_prompt(prompt):
    predictions = [classify_article_with_prompt(prompt, article_text) for article_text in train_dataset['text']]
    return accuracy_score(train_dataset['label'], predictions)

# Bayesian optimization function
def black_box_function(prompt_idx):
    prompt = enhanced_prompts[int(prompt_idx)]
    return evaluate_prompt(prompt)

pbounds = {'prompt_idx': (0, len(enhanced_prompts) - 1)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(
    init_points=5,
    n_iter=15,
)

|   iter    |  target   | prompt... |
-------------------------------------
| 1         | 0.9929    | 50.19     |
| 2         | 0.9942    | 127.4     |
| 3         | 0.9929    | 98.09     |
| 4         | 0.9929    | 80.22     |
| 5         | 0.9942    | 20.91     |
| 6         | 0.9936    | 6.648     |
| 7         | 0.9929    | 126.3     |
| 8         | 0.9942    | 48.89     |
| 9         | 0.9942    | 127.4     |
| 10        | 0.9942    | 48.18     |
| 11        | 0.9949    | 19.97     |
| 12        | 0.9949    | 19.16     |
| 13        | 0.9923    | 17.89     |
| 14        | 0.9929    | 46.64     |
| 15        | 0.9923    | 22.41     |
| 16        | 0.9929    | 129.2     |
| 17        | 0.9949    | 19.55     |
| 18        | 0.9942    | 64.38     |
| 19        | 0.9929    | 63.05     |
| 20        | 0.9936    | 65.55     |


In [8]:
max_target_value = max(res['target'] for res in optimizer.res)

best_prompts_dict = {}


for res in optimizer.res:
    if res['target'] == max_target_value:
        idx = int(res['params']['prompt_idx'])  
        prompt = enhanced_prompts[idx]          
        best_prompts_dict[idx] = prompt         


print("Best prompt indices and their prompts:", best_prompts_dict)


Best prompt indices and their prompts: {19: 'In which section of the newspaper is this news story most likely to appear?'}


In [9]:
# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments setup
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed=42,  # Ensure reproducibility
)

# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Iterate over each of the best prompts
for idx, best_prompt in best_prompts_dict.items():
    print(f"Training and evaluating for best prompt index: {idx}")
    print(f"Best prompt: {best_prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{best_prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer2 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt,
        eval_dataset=val_data_with_prompt,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer2.train()

    # Evaluate the model on the test set
    eval_result = trainer2.evaluate(test_data_with_prompt)
    print(f"Final evaluation results for prompt index {idx} on test set: {eval_result}")

Training and evaluating for best prompt index: 19
Best prompt: In which section of the newspaper is this news story most likely to appear?


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.197627,0.977578,0.977580,0.978229,0.977578
2,No log,0.223333,0.959641,0.959780,0.960610,0.959641
3,No log,0.247450,0.968610,0.968514,0.968621,0.968610
4,No log,0.231031,0.973094,0.973006,0.973118,0.973094
5,No log,0.302427,0.964126,0.964094,0.964695,0.964126
6,0.022500,0.283280,0.964126,0.964094,0.964695,0.964126
7,0.022500,0.305275,0.964126,0.964094,0.964695,0.964126
8,0.022500,0.268565,0.973094,0.973006,0.973118,0.973094
9,0.022500,0.318484,0.959641,0.959536,0.960497,0.959641
10,0.022500,0.288549,0.959641,0.959536,0.960497,0.959641


Final evaluation results for prompt index 19 on test set: {'eval_loss': 0.130619615316391, 'eval_accuracy': 0.9707865168539326, 'eval_f1': 0.9710495530515991, 'eval_precision': 0.9719055772844216, 'eval_recall': 0.9707865168539326, 'eval_runtime': 4.5629, 'eval_samples_per_second': 97.526, 'eval_steps_per_second': 6.136, 'epoch': 10.0}


In [10]:
# classify function
def classify_article_with_prompt(prompt, article_text):
    input_text = f"{prompt} {article_text}"
    inputs = tokenizer(input_text, truncation=True, max_length=512, padding='max_length', return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    label_id = outputs.logits.argmax(dim=1).item()
    return label_id

# evaluation function
def evaluate_prompt(prompt):
    predictions = [classify_article_with_prompt(prompt, article_text) for article_text in train_dataset['text']]
    return accuracy_score(train_dataset['label'], predictions)

# Bayesian optimization function
def black_box_function(prompt_idx):
    prompt = enhanced_prompts1[int(prompt_idx)]
    return evaluate_prompt(prompt)

pbounds = {'prompt_idx': (0, len(enhanced_prompts1) - 1)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(
    init_points=5,
    n_iter=15,
)

|   iter    |  target   | prompt... |
-------------------------------------
| 1         | 0.9929    | 35.58     |
| 2         | 0.9505    | 90.32     |
| 3         | 0.9884    | 69.54     |
| 4         | 0.9936    | 56.87     |
| 5         | 0.9949    | 14.82     |
| 6         | 0.9936    | 0.0       |
| 7         | 0.9942    | 16.5      |
| 8         | 0.9917    | 46.08     |
| 9         | 0.9955    | 6.882     |
| 10        | 0.9949    | 27.71     |
| 11        | 0.9929    | 62.39     |
| 12        | 0.9949    | 10.26     |
| 13        | 0.9949    | 3.878     |
| 14        | 0.9949    | 30.91     |
| 15        | 0.9936    | 52.01     |
| 16        | 0.9955    | 6.014     |
| 17        | 0.9955    | 6.444     |
| 18        | 0.9955    | 6.447     |
| 19        | 0.9955    | 6.45      |
| 20        | 0.9955    | 6.449     |


In [11]:
max_target_value = max(res['target'] for res in optimizer.res)

best_prompts_dict = {}


for res in optimizer.res:
    if res['target'] == max_target_value:
        idx = int(res['params']['prompt_idx'])  
        prompt = enhanced_prompts1[idx]          
        best_prompts_dict[idx] = prompt         


print("Best prompt indices and their prompts:", best_prompts_dict)

Best prompt indices and their prompts: {6: 'Which category is most appropriate for this news article?'}


In [12]:
# Training arguments setup
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed=42,  # Ensure reproducibility
)

# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

for idx, best_prompt in best_prompts_dict.items():
    print(f"Training and evaluating for best prompt index: {idx}")
    print(f"Best prompt: {best_prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{best_prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt1 = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt1 = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt1 = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer3 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt1,
        eval_dataset=val_data_with_prompt1,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer3.train()

    # Evaluate the model on the test set
    eval_result = trainer3.evaluate(test_data_with_prompt)
    print(f"Final evaluation results for prompt index {idx} on test set: {eval_result}")

Training and evaluating for best prompt index: 6
Best prompt: Which category is most appropriate for this news article?


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.246820,0.959641,0.959780,0.960610,0.959641
2,No log,0.258069,0.964126,0.964150,0.965317,0.964126
3,No log,0.254338,0.964126,0.964094,0.964695,0.964126
4,No log,0.214135,0.977578,0.977491,0.977789,0.977578
5,No log,0.310939,0.955157,0.955118,0.956920,0.955157
6,0.014000,0.270651,0.968610,0.968380,0.968806,0.968610
7,0.014000,0.257252,0.968610,0.968383,0.968439,0.968610
8,0.014000,0.258074,0.968610,0.968383,0.968439,0.968610
9,0.014000,0.296987,0.959641,0.959616,0.960731,0.959641
10,0.014000,0.288854,0.959641,0.959616,0.960731,0.959641


Final evaluation results for prompt index 6 on test set: {'eval_loss': 0.1575334519147873, 'eval_accuracy': 0.9775280898876404, 'eval_f1': 0.9776256279036483, 'eval_precision': 0.9781581434421927, 'eval_recall': 0.9775280898876404, 'eval_runtime': 4.5569, 'eval_samples_per_second': 97.653, 'eval_steps_per_second': 6.144, 'epoch': 10.0}


In [13]:
compareprompts = ["What is the central focus of this newspaper article?",
"How would you formulate the main theme of this message?"]

compareprompts1 =  ["What category best fits this news article?",
"How would you describe the main topic of this news?",
"In which section of the newspaper would this news likely be found?"]


In [14]:
compareprompts = ["What is the central focus of this newspaper article?",
                  "How would you formulate the main theme of this message?"]

# Training arguments setup
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",  # Ensure reproducibility
    seed=42
)

# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluate each prompt from compareprompts
for prompt in compareprompts:
    print(f"Evaluating with prompt: {prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt2 = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt2 = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt2 = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer4 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt,
        eval_dataset=val_data_with_prompt,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer4.train()

    # Evaluate the model on the test set
    eval_result = trainer4.evaluate(test_data_with_prompt)
    print(f"Final evaluation results for prompt '{prompt}' on test set: {eval_result}")


Evaluating with prompt: What is the central focus of this newspaper article?


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.381782,0.959641,0.959687,0.960532,0.959641
2,No log,0.239162,0.968610,0.968420,0.968415,0.968610
3,No log,0.350070,0.968610,0.968556,0.969626,0.968610
4,No log,0.367956,0.973094,0.972998,0.973309,0.973094
5,No log,0.373187,0.959641,0.959616,0.960731,0.959641
6,0.004700,0.341250,0.955157,0.955249,0.956721,0.955157
7,0.004700,0.374926,0.964126,0.964107,0.964496,0.964126
8,0.004700,0.475549,0.955157,0.955249,0.956721,0.955157
9,0.004700,0.452364,0.955157,0.955249,0.956721,0.955157
10,0.004700,0.427626,0.955157,0.955249,0.956721,0.955157


Final evaluation results for prompt 'How would you formulate the main theme of this message?' on test set: {'eval_loss': 0.21203429996967316, 'eval_accuracy': 0.9752808988764045, 'eval_f1': 0.9753541440834665, 'eval_precision': 0.9757718817864225, 'eval_recall': 0.9752808988764045, 'eval_runtime': 4.5372, 'eval_samples_per_second': 98.079, 'eval_steps_per_second': 6.171, 'epoch': 10.0}


In [15]:
# Training arguments setup
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",  # Ensure reproducibility
    seed=42
)

# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluate each prompt from compareprompts
for prompt in compareprompts1:
    print(f"Evaluating with prompt: {prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt3 = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt3 = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt3 = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer5 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt3,
        eval_dataset=val_data_with_prompt3,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer5.train()

    # Evaluate the model on the test set
    eval_result = trainer5.evaluate(test_data_with_prompt3)
    print(f"Final evaluation results for prompt '{prompt}' on test set: {eval_result}")


Evaluating with prompt: What category best fits this news article?


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.563040,0.959641,0.959481,0.959633,0.959641
2,No log,0.441742,0.964126,0.964019,0.964112,0.964126
3,No log,0.564115,0.964126,0.963938,0.963949,0.964126
4,No log,0.543142,0.964126,0.963938,0.963949,0.964126
5,No log,0.528898,0.968610,0.968383,0.968439,0.968610
6,0.004800,0.523051,0.968610,0.968383,0.968439,0.968610
7,0.004800,0.524130,0.968610,0.968383,0.968439,0.968610
8,0.004800,0.525091,0.968610,0.968383,0.968439,0.968610
9,0.004800,0.526065,0.968610,0.968383,0.968439,0.968610
10,0.004800,0.534003,0.968610,0.968383,0.968439,0.968610


Final evaluation results for prompt 'In which section of the newspaper would this news likely be found?' on test set: {'eval_loss': 0.30128052830696106, 'eval_accuracy': 0.9775280898876404, 'eval_f1': 0.9774765351877087, 'eval_precision': 0.9778827370814919, 'eval_recall': 0.9775280898876404, 'eval_runtime': 4.5192, 'eval_samples_per_second': 98.47, 'eval_steps_per_second': 6.196, 'epoch': 10.0}
